<a href="https://colab.research.google.com/github/waterta0115/LDA_graduattion_thesis/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import japanize_matplotlib
import arviz as az
import matplotlib.dates as mdates
import igraph as ig
import warnings
warnings.filterwarnings('ignore')
import logging
from tqdm import tqdm
from gensim.models.coherencemodel import CoherenceModel
az.style.use("arviz-doc")


# フィルタクラスを定義
class TopicLogFilter(logging.Filter):
    def filter(self, record):
        # メッセージに "topic #" が含まれていたら False（表示しない）を返す
        return "topic #" not in record.getMessage()
class ExcludeAlphaEtaFilter(logging.Filter):
    def filter(self, record):
        msg = record.getMessage()
        if "alpha" in msg or "eta" in msg:
            return False  # このログは表示しない
        return True       # それ以外は表示する

# ルートロガーにフィルター追加
logger = logging.getLogger()
logger.addFilter(ExcludeAlphaEtaFilter())

logger = logging.getLogger('gensim.models.ldamodel')
logger.addFilter(TopicLogFilter())

In [ ]:
from __future__ import absolute_import, division, unicode_literals  # noqa
import logging
import sys
import lda._lda
import lda.utils
from scipy.special import digamma, polygamma

# ログ出力の設定(1)
logger = logging.getLogger('lda')

class LDA:
    def __init__(self, n_topics, n_iter=2000, update_alpha=False,update_eta=False,alpha=0.1, eta=0.01, random_state=None,
                 refresh=10):
        self.n_topics = n_topics
        self.n_iter = n_iter
        self.update_alpha = update_alpha
        self.alpha = alpha
        self.update_eta = update_eta
        self.eta = eta

        # if random_state is None, check_random_state(None) does nothing
        # other than return the current numpy RandomState
        self.random_state = random_state
        self.refresh = refresh

        alpha_is_valid = alpha > 0 if isinstance(alpha, (int, float)) else np.all(alpha > 0)
        if not alpha_is_valid or eta <= 0:
             raise ValueError("alpha and eta must be greater than zero")

        # random numbers that are reused
        rng = lda.utils.check_random_state(random_state)
        self._rands = rng.rand(1024**2 // 8)  # 1MiB of random variates

        # configure console logging if not already configured
        # ログ出力の設定(2)
        if len(logger.handlers) == 1 and isinstance(logger.handlers[0], logging.NullHandler):
            logging.basicConfig(level=logging.INFO)

    def fit(self, X, y=None):
        self._fit(X)
        return self

    def fit_transform(self, X, y=None):
        if isinstance(X, np.ndarray):
            # in case user passes a (non-sparse) array of shape (n_features,)
            # turn it into an array of shape (1, n_features)
            X = np.atleast_2d(X)
        self._fit(X)
        return self.doc_topic_

    def transform(self, X, max_iter=20, tol=1e-16):
        if isinstance(X, np.ndarray):
            # in case user passes a (non-sparse) array of shape (n_features,)
            # turn it into an array of shape (1, n_features)
            X = np.atleast_2d(X)
        doc_topic = np.empty((X.shape[0], self.n_topics))
            # 結果格納用の配列を用意
        WS, DS = lda.utils.matrix_to_lists(X)
            # 行列→リスト　WS：単語インデックス　DS：その単語がどの文書に属しているか　に変換
        """
        Doc1: 単語1が2回, 単語3が1回
        Doc2: 単語2が3回
        """
        # この場合
        # WS = [1, 1, 3, 2, 2, 2]
        # DS = [0, 0, 0, 1, 1, 1]
        # このようにWSとDSは同じ長さとなる

        # TODO: this loop is parallelizable
        for d in np.unique(DS):
            doc_topic[d] = self._transform_single(WS[DS == d], max_iter, tol)
        return doc_topic

    def _transform_single(self, doc, max_iter, tol):
        # 内部的に doc = WS[DS == d] のような変換が行われている(あるdに属している単語集合)
        # ここでdocは「文書中の単語たち」を表す 単語インデックスの配列
        PZS = np.zeros((len(doc), self.n_topics))
        for iteration in range(max_iter + 1):  # +1 is for initialization
            PZS_new = self.components_[:, doc].T
                # components_はトピック-単語分布行列(phi)
                # その中からdoc内に含まれている単語だけを抽出して転置
                # PZS_newは各単語が「どのトピックにどれくらい関係しているか」の確率表
            PZS_new *= (PZS.sum(axis=0) - PZS + self.alpha)
                # PZS.sum(axis=0)：各文書はどれくらいの数のトピックを持っているか
                    # ここではブロードキャスト機能が自動的に働くため、PZSの次元は(1,n_topics)となる
                # PZS.sum(axis=0)-PZS：shape==(n_words_in_doc,n_topics)
                # self.alphaはスカラー又は(n_topics,)であるが、ブロードキャストにより全体の次元は(n_words_in_doc,n_topics)
            PZS_new /= PZS_new.sum(axis=1)[:, np.newaxis]  # vector to single column matrix
                # np.array([a,b,c])[:,np.newaxis]とすることで1次元配列を2次元配列にすることが可能
                    # (3,)→(3,1)；これらは全くの別物
                # ここでは確率にしている（正規化）
            delta_naive = np.abs(PZS_new - PZS).sum()
                # 前回の値との変化量
            logger.debug('transform iter {}, delta {}'.format(iteration, delta_naive))
            PZS = PZS_new
            if delta_naive < tol:
                break
                # 収束判定
        theta_doc = PZS.sum(axis=0) / PZS.sum()
            # 文書内のすべての単語にわたるトピック確率を合計し、全体で正規化して文書レベルのトピック分布を作る
        assert len(theta_doc) == self.n_topics
        assert theta_doc.shape == (self.n_topics,)
        return theta_doc

    def _fit(self, X):
        """inference by """
        random_state = lda.utils.check_random_state(self.random_state)
        rands = self._rands.copy()
            # randsは事前に生成された乱数列のコピー
            # gibbs samplingで使う乱数をここから供給する
        self._initialize(X)
            # 各単語にランダムにトピックを割り当てる
            # 各カウント行列を作成：nzw_,ndz_,nz_
                # nzw_：topic-wordの共起数
                # ndz_：doc-topicの共起数
                # nz_：各トピックに属する単語数の合計
        for it in range(self.n_iter):
            # FIXME: using numpy.roll with a random shift might be faster
            random_state.shuffle(rands)
            if it % self.refresh == 0:
                ll = self.loglikelihood()
                logger.info("<{}> log likelihood: {:.0f}".format(it, ll))
                # keep track of loglikelihoods for monitoring convergence
                self.loglikelihoods_.append(ll)
            self._sample_topics(rands)
            if it == 1:
                print("ndz 初回サンプル:",self.ndz_[:5])
            if it == 100:
                print("ndz 100回目:", self.ndz_[:5])
            if self.update_alpha:     # ユーザが制御できるようにオプション化
                self._update_alpha()
                if it % 10 == 0:
                    logger.info(f"iteration{it}: alpha = {self.alpha}")
            if self.update_eta:     # ユーザが制御できるようにオプション化
                self._update_eta()
                if it % 10 == 0:
                    logger.info(f"iteration{it}: alpha = {self.eta}")
            # ここがcollapsed gibbs samplingの実体。lda-projectの内部で構築されている
        ll = self.loglikelihood()
            # 最終的な対数尤度の計算・出力
        logger.info("<{}> log likelihood: {:.0f}".format(self.n_iter - 1, ll))
        # note: numpy /= is integer division
        self.components_ = (self.nzw_ + self.eta).astype(float)
            # components_とは各トピックごとの単語分布（phi）：(n_topics,n_words)
            # etaを加えることでスムージングしている
        self.components_ /= np.sum(self.components_, axis=1)[:, np.newaxis]
            # 確率にしている（正規化）
        self.topic_word_ = self.components_
            # scikit-learn互換用のために同じ内容を別名で保存
        self.doc_topic_ = (self.ndz_ + self.alpha).astype(float)
            # doc_topic_とは各文書ごとのトピック分布(theta)：(n_docs,n_topics)
        self.doc_topic_ /= np.sum(self.doc_topic_, axis=1)[:, np.newaxis]

        # delete attributes no longer needed after fitting to save memory and reduce clutter
        del self.WS
        del self.DS
        del self.ZS
        return self
            # components_とdoc_topic_とloglikelihoods_を返す

    def _initialize(self, X):
        D, W = X.shape
        N = int(X.sum())
        n_topics = self.n_topics
        n_iter = self.n_iter
        logger.info("n_documents: {}".format(D))
        logger.info("vocab_size: {}".format(W))
        logger.info("n_words: {}".format(N))
        logger.info("n_topics: {}".format(n_topics))
        logger.info("n_iter: {}".format(n_iter))

        self.nzw_ = nzw_ = np.zeros((n_topics, W), dtype=np.intc, order="F")
        self.ndz_ = ndz_ = np.zeros((D, n_topics), dtype=np.intc, order="C")
        self.nz_ = nz_ = np.zeros(n_topics, dtype=np.intc)

        self.WS, self.DS = WS, DS = lda.utils.matrix_to_lists(X)
        self.ZS = ZS = np.empty_like(self.WS, dtype=np.intc)
        np.testing.assert_equal(N, len(WS))
        for i in range(N):
            w,d = WS[i], DS[i]
            z_new = i % n_topics
            ZS[i] = z_new
            ndz_[d, z_new] += 1
            nzw_[z_new, w] += 1
            nz_[z_new] += 1
        self.loglikelihoods_ = []

    def loglikelihood(self):# ここではalphaやetaはスカラーであることが想定されている
        nzw, ndz, nz = self.nzw_, self.ndz_, self.nz_
        alpha = self.alpha
        eta = self.eta
        nd = np.sum(ndz, axis=1).astype(np.intc)
        if isinstance(alpha, np.ndarray) and alpha.ndim > 0:
            alpha_scalar = alpha[0]
        else:
            alpha_scalar = alpha
        if isinstance(eta, np.ndarray) and eta.ndim > 0:
            eta_scalar = eta[0]
        else:
            eta_scalar = eta
        return lda._lda._loglikelihood(nzw, ndz, nz, nd, alpha_scalar, eta_scalar)

    def _sample_topics(self, rands):# ここではalphaやetaはK次元であることが想定されている
        n_topics, vocab_size = self.nzw_.shape
        if isinstance(self.alpha, (int, float)):
            alpha_array = np.full(n_topics, self.alpha)
        else:
            alpha_array = self.alpha
        if isinstance(self.eta, (int, float)):
            eta_array = np.full(n_topics, self.eta)
        else:
            eta_array = self.eta
        alpha = alpha_array.astype(np.float64)
        eta = eta_array.astype(np.float64)
        lda._lda._sample_topics(self.WS, self.DS, self.ZS, self.nzw_, self.ndz_, self.nz_,
                                alpha, eta, rands)

    import numpy as np
    from scipy.special import digamma, polygamma
    def _update_alpha(self):
        if not hasattr(self, "ndz_"):
            raise ValueError("ndz_ が存在しません")

        if np.all(self.ndz_ == self.ndz_[0]):
            print("警告: ndz_ が全ドキュメント同じ値になっています")

        dt = self.ndz_              # shape (D, K)
        D, K = dt.shape

        # ensure alpha is scalar
        if isinstance(self.alpha, np.ndarray):
            alpha_scalar = float(self.alpha[0])
        else:
            alpha_scalar = float(self.alpha) # float型の場合はそのままfloatに変換

        # s = digamma(dt + alpha_scalar).sum(axis=0)
        # g = s - D * digamma(alpha_scalar)
        # h = -D * polygamma(1, alpha_scalar)
        # alpha_new = alpha_scalar - g / h
        for i in range(100):
            N_d = np.sum(dt,axis=1)
            term1_num = np.sum(digamma(dt+alpha_scalar))
            term2_num = D*K*digamma(alpha_scalar)
            numerator = term1_num - term2_num
            term1_den = K*np.sum(digamma(N_d+alpha_scalar*K))
            term2_den = D*K*digamma(alpha_scalar*K)
            denominator = term1_den - term2_den
            alpha_new = alpha_scalar*(numerator/denominator)
            diff = np.abs(alpha_new-alpha_scalar)
            print(f"Iter{i+1}:alpha={alpha_new:.6f}(diff={diff:.6e})")
            if diff<1e-05:
                print("収束しました")
                break
            alpha_scalar = alpha_new
        # store symmetric vector
        self.alpha = np.ones(K)*alpha_new

        # logging
        logger.info(f"alpha updated (symmetric): {alpha_new}")

        if not hasattr(self, "alpha_history"):
            self.alpha_history = []
        self.alpha_history.append(self.alpha.copy())
    def _update_eta(self):
        if not hasattr(self, "nzw_"):
            raise ValueError("nzw_ が存在しません")

        nzw = self.nzw_              # shape (K, V)
        nz = self.nz_               # shape (K,)
        K, V = nzw.shape

        # ensure eta is scalar
        if isinstance(self.eta, np.ndarray):
            eta_scalar = float(self.eta[0])
        else:
            eta_scalar = float(self.eta)

        # fixed-point iteration (symmetric eta)
        for i in range(100):
            term1_num = np.sum(digamma(nzw + eta_scalar))     # Σ_k Σ_w ψ(n_kw + η)
            term2_num = K * V * digamma(eta_scalar)           # K*V ψ(η)
            numerator = term1_num - term2_num

            term1_den = V * np.sum(digamma(nz + V * eta_scalar))  # V Σ_k ψ(n_k· + Vη)
            term2_den = K * V * digamma(V * eta_scalar)           # K*V ψ(Vη)
            denominator = term1_den - term2_den

            eta_new = eta_scalar * (numerator / denominator)
            diff = abs(eta_new - eta_scalar)

            print(f"[eta] Iter{i+1}: eta={eta_new:.6f} (diff={diff:.6e})")

            if diff < 1e-5:
                print("eta 収束しました")
                break

            eta_scalar = eta_new

        # store symmetric eta vector
        self.eta = eta_new

        # logging
        logger.info(f"eta updated (symmetric): {eta_new}")

        # store history
        if not hasattr(self, "eta_history"):
            self.eta_history = []
        self.eta_history.append(eta_new)

In [ ]:
def prepare(df):
    from gensim import corpora
    texts = df.groupby("文書")["単語"].apply(list).tolist()
    filtered_texts = [doc for doc in texts if len(doc) > 2]
    dictionary = corpora.Dictionary(filtered_texts)
    corpus = [dictionary.doc2bow(text) for text in filtered_texts]
    return filtered_texts,dictionary,corpus

In [ ]:
def make_bow(data):
    from sklearn.feature_extraction.text import CountVectorizer
    import pandas as pd

    doc_word_lists = data.groupby("文書")["単語"].apply(list)

    filtered_doc_lists = doc_word_lists[doc_word_lists.apply(len) > 2]

    docs = filtered_doc_lists.apply(lambda items: ' '.join(map(str, items)))

    vectorizer = CountVectorizer(token_pattern='[^ ]+')
    bow_matrix = vectorizer.fit_transform(docs)
    vocab = vectorizer.get_feature_names_out()

    # n_samples, n_features
    print(f"元の文書数（ユニーク）：{len(doc_word_lists)}")
    print(f"フィルタリング後の文書数：{len(filtered_doc_lists)}")
    print("BOW shape:", bow_matrix.shape)

    return bow_matrix, vocab

In [ ]:
# Perplexity算出のためにトレインテストスプリットを施したもの
def make_bow_split(data):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import train_test_split
    import pandas as pd

    doc_word_lists = data.groupby("文書")["単語"].apply(list)

    filtered_doc_lists = doc_word_lists[doc_word_lists.apply(len) > 2]

    docs = filtered_doc_lists.apply(lambda items: ' '.join(map(str, items)))

    vectorizer = CountVectorizer(token_pattern='[^ ]+')
    bow_matrix = vectorizer.fit_transform(docs)
    vocab = vectorizer.get_feature_names_out()
    X_train, X_test = train_test_split(bow_matrix, test_size=0.2, random_state=42)

    # n_samples, n_features
    print(f"元の文書数（ユニーク）：{len(doc_word_lists)}")
    print(f"フィルタリング後の文書数：{len(filtered_doc_lists)}")
    print("BOW shape:", bow_matrix.shape)

    return X_train,X_test,vocab

In [ ]:
from tqdm import tqdm
# Xにテストデータを入れる
def calculate_perplexity_fast(model, X):
    # 1. テストデータのトピック分布を推論 (n_docs, n_topics)
    doc_topic_dist = model.transform(X)

    # 2. トピック-単語分布を正規化 (n_topics, n_words)
    topic_word_dist = model.components_ / model.components_.sum(axis=1)[:, np.newaxis]

    doc_word_prob = np.dot(doc_topic_dist, topic_word_dist)

    epsilon = 1e-10

    if hasattr(X, "toarray"):
        X_arr = X.toarray()
    else:
        X_arr = X

    # 要素ごとの計算
    log_likelihood = np.sum(X_arr * np.log(doc_word_prob + epsilon))

    total_words = X.sum()
    perplexity = np.exp(-log_likelihood / total_words)

    return perplexity

# 実行

In [ ]:
train,test,vocab = make_bow(data)
texts,dictionary,corpus = prepare(data)

In [ ]:
# ハイパラ更新なしでperplexity測る用にLDAを実行
start = 2
limit = 30
step = 1
perplexity_vals = []
from tqdm import tqdm
for n_topic in tqdm(range(start, limit, step), desc="Calculating perplexity"):
    lda = LDA(n_topic,alpha=0.1,eta=0.01,update_alpha=False,update_eta=False, random_state = 0, n_iter = 1500)
    lda.fit(train)
    perplexity=calculate_perplexity_fast(lda, test)
    perplexity_vals.append(perplexity)

In [ ]:
plt.figure(figsize=(10, 6))
topic_range = range(2, 16, 1)
plt.plot(topic_range, perplexity_vals, marker='o')

plt.title("テストデータに対するPerplexity",fontsize=25)
plt.xlabel("トピック数 (K)",fontsize=20)
plt.ylabel("Perplexity",fontsize=20)
plt.grid(True)
plt.tick_params(axis='x', labelsize=20)  # 横軸
plt.tick_params(axis='y', labelsize=20)
plt.savefig("/home/jovyan/fig/perp_loyal.pdf",dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 例えばトピックの候補８においてLDAを実行する
lda8 = LDA(n_topics = 8,alpha=0.1,eta=0.01,update_alpha=True,update_eta=True, random_state = 0, n_iter = 1500)
lda8.fit(train)
# 他にも候補点においてLDAを実行する。

NameError: name 'LDA' is not defined

In [ ]:
# ハイパラが収束しているのかの確認
start = 1000
end = 1500
# 平均値を計算
eta_mean = np.mean(lda8.eta_history[start:end])
fig, ax = plt.subplots(figsize=(12, 6))

# alpha history
ax.plot(lda8.eta_history, label="eta history")

# 平均線（1000〜1999）
ax.axhline(eta_mean, linestyle='--', alpha=0.7,
           label=f"mean (1000-1500) = {eta_mean:.4f}")

# タイトル・軸
ax.set_title("Eta History of loyal", fontsize=24)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.grid(True, linestyle='--', alpha=0.5)

ax.legend(fontsize=17)

plt.tight_layout()
plt.savefig("/home/jovyan/fig/loyal8_eta_history.png",dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Coherenceを計測
topic_word_lda8 = lda8.topic_word_   # shape (n_topics, vocab)
topics_lda8 = []
n_top_words = 5
n_topic = 8
coherence_vals=[]
for t in range(n_topic):
    # topn_uni = topic_word_uni[t].argsort()[:-n_top_words - 1:-1]
    # topics_uni.append([vocab_uni[w_id] for w_id in topn_uni])
    word_ids = topic_word_lda8[t].argsort()[-n_top_words:][::-1]
    word_ids = [int(i) for i in word_ids]  # numpy型→python int
    words = [vocab[i] for i in word_ids]
    topics_lbr8.append(words)
cm = CoherenceModel(topics=topics_lda8, texts=texts, dictionary=dictionary, coherence='c_npmi')
coherence_vals.append(cm.get_coherence())

In [ ]:
# 候補8,10,12,14でのcoherenceの可視化（比較）
x = [8,10,12,14]
c1 = 'darkturquoise'
plt.plot(x, coherence_vals, 'o-', color=c1)
plt.xlabel('トピック数(K)',fontsize=25)
plt.ylabel('Coherence(NPMI)',fontsize=25)
plt.tick_params(axis='x', labelsize=20)  # 横軸
plt.tick_params(axis='y', labelsize=20)  # 縦軸
plt.legend(fontsize=20)
plt.grid(True, linestyle='--', alpha=0.5)
plt.savefig("/home/jovyan/fig/coherence_loyal.pdf",dpi=300, bbox_inches='tight')
plt.show()

Coherenceで最も値が大きいのが最適トピック数となる。最適トピック数は10であったと仮定して、10でLDAを実行する。

In [ ]:
lda10 = LDA(n_topics = 8,alpha=0.1,eta=0.01,update_alpha=False,update_eta=False, random_state = 0, n_iter = 1500)
lda10.fit(train)
# alphaとetaの値は学習・更新した際の値を使う。

In [ ]:
import pyLDAvis

tt_10 = lda10.topic_word_   # shape: (n_topics, n_terms) term distribution
dt_10 = lda10.doc_topic_     # shape: (n_docs, n_topics) topic distribution
term_frequency = np.asarray(train.sum(axis=0)).flatten()  # 各単語の出現回数

# --- pyLDAvis形式に変換して可視化 ---
vis_data_10 = pyLDAvis.prepare(
    topic_term_dists=tt_10,
    doc_topic_dists=dt_10,
    doc_lengths=train.sum(axis=1).A1,
    vocab=vocab,
    term_frequency=term_frequency
)
pyLDAvis.display(vis_data_10)

In [ ]:
# 単語分布の保存
column_names = ['topic1',"topic2","topic3","topic4","topic5","topic6",
               "topic7","topic8","topic9","topic10"]

tt_df = pd.DataFrame(
    data=tt_10,
    index=column_names,
    columns=vocab
)
tt_df.to_csv("h2o/matrix/tt_loyal.csv", encoding='utf-8', index=True)

In [ ]:
# トピック分布の保存
matrix_length = dt_10.shape[0]
new_index_list = list(range(matrix_length))

dt_df = pd.DataFrame(
    data=dt_10,
    index=new_index_list,
    columns=column_names
)
dt_df.to_csv("h2o/matrix/dt_loyal.csv", encoding='utf-8', index=False)

# ネットワーク分析

## 準備

In [ ]:
def adj_matrix_dt(topic_term_dists):
    matrix = np.dot(topic_term_dists.T, topic_term_dists)
    if matrix.max() != matrix.min():
        matrix = (matrix - matrix.min()) / (matrix.max() - matrix.min())
    return matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def find_threshold_by_count(matrix, target_edge_count):
    """
    エッジの重み行列から、指定した「残したい本数」になるような閾値を計算し、
    CDF（累積分布関数）とともに可視化する関数。

    Args:
        matrix (np.ndarray): 接続行列（隣接行列）
        target_edge_count (int): 最終的に残したいエッジの本数

    Returns:
        tau (float): 計算された閾値
    """

    # --- 1. データ抽出 ---
    # 上三角行列（対角成分除く）からインデックスを取得し、重複を避ける
    indices = np.triu_indices(len(matrix), k=1)
    edge_weights = matrix[indices]

    # 0より大きい（実際に存在する）エッジの重みだけを抽出
    existing_edges = edge_weights[edge_weights > 0]

    if len(existing_edges) == 0:
        print("エラー: グラフにエッジが存在しません。")
        return 0.0

    # --- 2. 閾値の逆算ロジック ---
    # エッジの重みを昇順（小さい順）にソート
    sorted_weights = np.sort(existing_edges)
    total_edges = len(existing_edges)

    # 目標本数が全エッジ数を超えている場合のガード処理
    if target_edge_count >= total_edges:
        print("注意: 指定された本数が全エッジ数以上です。すべてのエッジを残します。")
        tau = 0.0 # すべて残すために0（または最小値より小さい値）にする
        # グラフ表示用のカット率
        calculated_cut_percentage = 0.0
    elif target_edge_count <= 0:
        print("注意: 指定された本数が0以下です。すべてのエッジを削除します。")
        tau = sorted_weights[-1] + 1.0 # 最大値より大きくする
        calculated_cut_percentage = 100.0
    else:
        # [小さい ...... 大きい] と並んでいる
        # 上位 N個 を残すには、後ろから N番目 の要素の手前を閾値にする
        # 例: 全100個, 残したい10個 -> index 0~89 (90個) をカット -> index 89 の値が閾値
        cutoff_index = total_edges - target_edge_count - 1

        # 閾値を決定（この値より大きいものを残す、とするのが一般的）
        tau = sorted_weights[cutoff_index]

        # カットされる割合を逆算（グラフ表示用）
        calculated_cut_percentage = (cutoff_index + 1) / total_edges * 100

    # --- 3. 結果の確認 ---
    # 実際にこの閾値でカットした場合に残る数を計算（同じ値が複数ある場合、ズレることがあるため確認）
    actual_kept_edges = np.sum(existing_edges > tau)

    print(f"--- 閾値計算結果 (Target Count Mode) ---")
    print(f"全エッジ数（0より大） : {total_edges}")
    print(f"残したい目標本数      : {target_edge_count}")
    print(f"決定された閾値 (τ)    : {tau:.10f}")
    print(f"---")
    print(f"実際に残るエッジ数    : {actual_kept_edges} (閾値 {tau:.10f} より大きいもの)")
    print(f"実際に残る割合        : {actual_kept_edges / total_edges * 100:.2f}%")
    print(f"計算上のカット率      : {calculated_cut_percentage:.2f}%")

    if actual_kept_edges != target_edge_count:
        print(f"※注意: 重みが同じエッジが存在するため、目標本数({target_edge_count})と完全に一致しませんでした。")

    # --- 4. 累積確率プロット（CDF）の作成と可視化 ---

    # Y軸（累積確率）を作成 (1/N, 2/N, ..., N/N)
    y_cumulative = np.arange(1, len(sorted_weights) + 1) / len(sorted_weights)

    plt.figure(figsize=(12, 7))
    # 累積確率プロット (CDF)
    plt.plot(sorted_weights, y_cumulative, color='blue', label='Cumulative Distribution (CDF)')

    # --- 5. 決定した閾値をグラフ上に表示 ---
    # 閾値 τ で垂直線を引く
    plt.axvline(x=tau, color='red', linestyle='--',
                label=f'Threshold τ = {tau:.10f}')

    # 累積確率 (カット率) の点で水平線を引く
    # y軸は 0.0 ~ 1.0 なので、パーセンテージを100で割る
    plt.axhline(y=calculated_cut_percentage / 100.0, color='red', linestyle=':',
                label=f'Cutoff Probability {calculated_cut_percentage/100:.2f} (Removes bottom {calculated_cut_percentage:.1f}%)')

    # 交点に印をつける
    plt.plot(tau, calculated_cut_percentage / 100.0, 'ro')

    plt.title(f'CDF of Edge Weights (Targeting top {target_edge_count} edges)',fontsize=20)
    plt.xlabel('Edge Weight (要素の値)',fontsize=20)
    plt.ylabel('Cumulative Probability (累積確率)',fontsize=20)
    plt.legend(fontsize=20)
    plt.grid(True, which="both", ls="--", alpha=0.5)

    # Y軸をパーセンテージ表示にする
    plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))

    plt.show()

    return tau

In [ ]:
import pandas as pd
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

def plot_topic_correlation_filtered(adj_matrix, topic_labels, file_path, threshold):
    """
    閾値以下のエッジを完全に削除して可視化する
    """
    matrix = adj_matrix.copy()
    np.fill_diagonal(matrix, 0)

    # 1. 正規化 (0-1)
    if matrix.max() != matrix.min():
        matrix = (matrix - matrix.min()) / (matrix.max() - matrix.min())

    # 2. 閾値以下の値を0にする（エッジを張らない）
    matrix[matrix < threshold] = 0

    # 3. グラフの作成（重みが0の箇所にはエッジが張られない）
    g = ig.Graph.Weighted_Adjacency(matrix.tolist(), mode="undirected")
    layout = g.layout("fr")

# --- 重なり回避のためのパラメータ設定 ---
    visual_style = {}

    # ① ノードラベルを外側に配置する設定
    visual_style["vertex_label"] = topic_labels
    visual_style["vertex_label_dist"] = 1.4      # ラベルをノードの中心から離す (重要)
    visual_style["vertex_label_degree"] = np.pi/4 # ラベルを出す方向 (真上なら -np.pi/2)
    visual_style["vertex_label_size"] = 20
    visual_style["vertex_size"] = 50

    # ② エッジラベル（数値）の視認性向上
    if g.ecount() > 0:
        visual_style["edge_label"] = [f"{w:.2f}" for w in g.es["weight"]]
        # エッジラベルを線の中央から少しずらす機能はigraphにはないため、
        # 色を薄くし、フォントサイズを下げることで「背景」化させます
        visual_style["edge_label_size"] = 15
        visual_style["edge_label_color"] = "blue"
        visual_style["edge_width"] = [w * 2 for w in g.es["weight"]]

    fig, ax = plt.subplots(figsize=(10, 10))
    ig.plot(g, layout=layout, target=ax, **visual_style)
    plt.savefig(file_path,dpi=300, bbox_inches='tight')
    plt.show()

## 実行

In [ ]:
dt_df_loyal = pd.read_csv("h2o/matrix/dt_loyal.csv")
dt_loyal= dt_df_loyal.to_numpy()

In [ ]:
adj_loyal = adj_matrix_dt(dt_loyal)
labels_loy = [f"Topic {i+1}" for i in range(adj_loyal.shape[0])]

In [ ]:
import pandas as pd
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

def plot_topic_correlation_filtered(adj_matrix, topic_labels, file_path, threshold):
    """
    閾値以下のエッジを完全に削除して可視化する
    """
    matrix = adj_matrix.copy()
    np.fill_diagonal(matrix, 0)

    # 1. 正規化 (0-1)
    if matrix.max() != matrix.min():
        matrix = (matrix - matrix.min()) / (matrix.max() - matrix.min())

    # 2. 閾値以下の値を0にする（エッジを張らない）
    matrix[matrix < threshold] = 0

    # 3. グラフの作成（重みが0の箇所にはエッジが張られない）
    g = ig.Graph.Weighted_Adjacency(matrix.tolist(), mode="undirected")
    layout = g.layout("fr")

# --- 重なり回避のためのパラメータ設定 ---
    visual_style = {}

    # ① ノードラベルを外側に配置する設定
    visual_style["vertex_label"] = topic_labels
    visual_style["vertex_label_dist"] = 1.4      # ラベルをノードの中心から離す (重要)
    visual_style["vertex_label_degree"] = np.pi/4 # ラベルを出す方向 (真上なら -np.pi/2)
    visual_style["vertex_label_size"] = 20
    visual_style["vertex_size"] = 50

    # ② エッジラベル（数値）の視認性向上
    if g.ecount() > 0:
        visual_style["edge_label"] = [f"{w:.2f}" for w in g.es["weight"]]
        # エッジラベルを線の中央から少しずらす機能はigraphにはないため、
        # 色を薄くし、フォントサイズを下げることで「背景」化させます
        visual_style["edge_label_size"] = 15
        visual_style["edge_label_color"] = "blue"
        visual_style["edge_width"] = [w * 2 for w in g.es["weight"]]

    fig, ax = plt.subplots(figsize=(10, 10))
    ig.plot(g, layout=layout, target=ax, **visual_style)
    plt.savefig(file_path,dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
plot_topic_correlation_filtered(adj_loyal,labels_loy,
                                file_path = "/home/jovyan/fig/network_dt_loyal.pdf",threshold = 0.23)

## 補足〜ブランドネットワークの構築〜

### 準備

In [ ]:
# 隣接行列の作成
def adj_matrix_(topic_term_dists,vocab,threshold):
    co_occurrence = np.dot(topic_term_dists.T, topic_term_dists)
    co_matrix = pd.DataFrame(co_occurrence, index=vocab, columns=vocab)
    threshold = np.quantile(co_matrix.values, threshold)
    filtered = co_matrix.mask(co_matrix < threshold, 0)
    adj_matrix = filtered.to_numpy(dtype=float)
    return adj_matrix

In [ ]:
def find_threshold(matrix,cut_percentage):
    # --- 2. 閾値計算のためのデータ抽出 ---
    import matplotlib.ticker as ticker

    # Φ (行列) から、0より大きいエッジの重みだけをすべて取り出す
    # (上三角行列または下三角行列のみを対象にし、重複カウントを避ける)
    indices = np.triu_indices(len(matrix), k=1) # k=1で対角成分を除く
    edge_weights = matrix[indices]

    # 0より大きい（実際に存在する）エッジの重みだけを抽出
    existing_edges = edge_weights[edge_weights > 0]

    if len(existing_edges) == 0:
        print("エラー: グラフにエッジが存在しません。")
    else:
        # --- 3. 閾値の決定 (アプローチ1) ---
        # np.percentile を使い、指定したパーセンタイルの値を計算する
        # これが求める閾値 τ (tau) となる
        # (95パーセンタイルの値 = 下から数えて95%地点の値)
        tau = np.percentile(existing_edges, cut_percentage)

        print(f"--- 閾値計算結果 ---")
        print(f"全エッジ数（0より大）: {len(existing_edges)}")
        print(f"弱いエッジを {cut_percentage}% カットする場合...")
        print(f"決定された閾値 (τ): {tau:.7f}")
        print(f"この閾値 τ より大きい（残すべき）エッジの数: {np.sum(existing_edges > tau)}")
        print(f"残るエッジの割合: {np.sum(existing_edges > tau) / len(existing_edges) * 100:.2f}%")


        # --- 4. 累積確率プロット（CDF）の作成と可視化 ---

        sorted_weights = np.sort(existing_edges)
        # Y軸（累積確率）を作成 (1/N, 2/N, ..., N/N)
        y_cumulative = np.arange(1, len(sorted_weights) + 1) / len(sorted_weights)

        plt.figure(figsize=(12, 7))
        # 累積確率プロット (CDF)
        plt.plot(sorted_weights, y_cumulative, color='blue', label='Cumulative Distribution (CDF)')

        # X軸を対数スケールにすると、べき乗則の「裾」が見やすくなる (オプション)
        # plt.xscale('log')

        # --- 5. 決定した閾値をグラフ上に表示 ---
        # 閾値 τ で垂直線を引く
        plt.axvline(x=tau, color='red', linestyle='--',
                    label=f'Threshold τ = {tau:.7f}')

        # 累積確率 cut_percentage/100 の点で水平線を引く
        plt.axhline(y=cut_percentage / 100.0, color='red', linestyle=':',
                    label=f'Cumulative Probability {cut_percentage / 100.0:.2f}')

        # 交点に印をつける
        plt.plot(tau, cut_percentage / 100.0, 'ro') # 'ro' = red circle marker

        plt.title('Cumulative Distribution Function (CDF) of Edge Weights')
        plt.xlabel('Edge Weight (要素の値)')
        plt.ylabel('Cumulative Probability (累積確率)')
        plt.legend()
        plt.grid(True, which="both", ls="--", alpha=0.5)

        # Y軸をパーセンテージ表示にすると見やすい
        plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0))

        plt.show()

In [ ]:
# ノードの次数に応じてノードサイズを変更
def plot_igraph_from_adjacency_deg(
    adjacency_matrix,
    labels,
    layout_type="fr",
    vertex_size=30,
    edge_width_scale=5,
    vertex_label_size=14,
    edge_curved=False,
    figsize=(8, 8),
    edge_threshold=0.0,
    vertex_color="skyblue",
    edge_color="gray",
    top_n_labels=20
):
    import igraph as ig
    import matplotlib.pyplot as plt
    import numpy as np

    # 隣接行列をnumpy配列に変換
    adj = np.array(adjacency_matrix)
    n = adj.shape[0]

    # エッジの重みが閾値以下のものは0にする
    adj = np.where(adj > edge_threshold, adj, 0)

    # igraphグラフ作成
    g = ig.Graph.Weighted_Adjacency(adj.tolist(), mode=ig.ADJ_UNDIRECTED, attr="weight", loops=False)
    g.vs["name"] = labels

    # 孤立ノード除去
    isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0.0]
    if isolated_nodes:
        g.delete_vertices(isolated_nodes)



    # ノードラベル
    # labels = vocab として渡される前提
    if "name" in g.vs.attributes():
        # g.vs["name"] から g.vs["label"] へコピー
        g.vs["label"] = g.vs["name"]
    else:
        # "name" がない場合のフォールバック
        g.vs["label"] = [str(i) for i in range(g.vcount())]
    # 次数の高いノードの上位top_n_labels件のみラベルを付与
    degrees = g.degree()
    top_indices = np.argsort(degrees)[-top_n_labels:]  # 上位top_n_labels件
    g.vs["label"] = [
        g.vs[i]["label"] if i in top_indices else "" for i in range(g.vcount())
    ]
    # レイアウト
    layout = g.layout(layout_type)

    # エッジの太さ
    edge_weights = g.es["weight"] if "weight" in g.es.attributes() else [1]*g.ecount()
    if len(edge_weights) > 0 and max(edge_weights) > 0:
        edge_widths = [edge_width_scale * (w / max(edge_weights)) for w in edge_weights]
    else:
        edge_widths = [1 for _ in edge_weights]

    # コミュニティ検出（例: Louvain法, fallback to fastgreedy if Louvain not available）
    try:
        communities = g.community_multilevel(weights=g.es['weight'] if 'weight' in g.es.attributes() else None)
    except AttributeError:
        # fallback method
        communities = g.community_fastgreedy(weights=g.es['weight'] if 'weight' in g.es.attributes() else None).as_clustering()
    membership = communities.membership
    import matplotlib
    cmap = matplotlib.cm.get_cmap('tab20')
    num_colors = cmap.N if hasattr(cmap, "N") else 20
    comm_vertex_colors = [matplotlib.colors.to_hex(cmap(i % num_colors)) for i in membership]
    g.vs["color"] = comm_vertex_colors
    # ノードの次数に応じてノードサイズを変更する
    degrees = g.degree()
    # 適度なスケーリングのために最小サイズと最大サイズを指定
    min_size = 20
    max_size = 80
    # スケーリング
    if degrees:
        min_deg = min(degrees)
        max_deg = max(degrees) if max(degrees) != min(degrees) else min(degrees) + 1
        # ノードサイズ算出
        scaled_sizes = [
            min_size + (deg - min_deg) / (max_deg - min_deg) * (max_size - min_size)
            for deg in degrees
        ]
    else:
        scaled_sizes = [min_size for _ in g.vs]
    g.vs["size"] = scaled_sizes

    # ノードサイズに基づき再描画（ノードの大きさ＝次数）
    fig, ax = plt.subplots(figsize=figsize)
    ig.plot(
        g,
        target=ax,
        layout=layout,
        vertex_size=g.vs["size"],
        vertex_label=g.vs["label"],
        vertex_label_size=vertex_label_size,
        vertex_color=g.vs["color"],
        edge_width=edge_widths,
        edge_color=edge_color,
        edge_curved=edge_curved,
        bbox=(figsize[0]*100, figsize[1]*100),
        margin=40,
    )
    plt.show()
    return g,communities

In [ ]:
# ノードの媒介中心性指標に応じてノードサイズを変更
def plot_igraph_from_adjacency_bet(
    adjacency_matrix,
    labels,
    layout_type="fr",
    vertex_size=30,
    edge_width_scale=5,
    vertex_label_size=14,
    edge_curved=False,
    figsize=(8, 8),
    edge_threshold=0.0,
    vertex_color="skyblue",
    edge_color="gray",
    top_n_labels=20
):
    import igraph as ig
    import matplotlib.pyplot as plt
    import numpy as np

    # 隣接行列をnumpy配列に変換
    adj = np.array(adjacency_matrix)
    n = adj.shape[0]

    # エッジの重みが閾値以下のものは0にする
    adj = np.where(adj > edge_threshold, adj, 0)

    # igraphグラフ作成
    g = ig.Graph.Weighted_Adjacency(adj.tolist(), mode=ig.ADJ_UNDIRECTED, attr="weight", loops=False)
    g.vs["name"] = labels

    # 孤立ノード除去
    isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0.0]
    if isolated_nodes:
        g.delete_vertices(isolated_nodes)



    # ノードラベル
    # labels = vocab として渡される前提
    if "name" in g.vs.attributes():
        # g.vs["name"] から g.vs["label"] へコピー
        g.vs["label"] = g.vs["name"]
    else:
        # "name" がない場合のフォールバック
        g.vs["label"] = [str(i) for i in range(g.vcount())]
    # 次数の高いノードの上位top_n_labels件のみラベルを付与
    degrees = g.betweenness()
    top_indices = np.argsort(degrees)[-top_n_labels:]  # 上位top_n_labels件
    g.vs["label"] = [
        g.vs[i]["label"] if i in top_indices else "" for i in range(g.vcount())
    ]
    # レイアウト
    layout = g.layout(layout_type)

    # エッジの太さ
    edge_weights = g.es["weight"] if "weight" in g.es.attributes() else [1]*g.ecount()
    if len(edge_weights) > 0 and max(edge_weights) > 0:
        edge_widths = [edge_width_scale * (w / max(edge_weights)) for w in edge_weights]
    else:
        edge_widths = [1 for _ in edge_weights]

    # コミュニティ検出（例: Louvain法, fallback to fastgreedy if Louvain not available）
    try:
        communities = g.community_multilevel(weights=g.es['weight'] if 'weight' in g.es.attributes() else None)
    except AttributeError:
        # fallback method
        communities = g.community_fastgreedy(weights=g.es['weight'] if 'weight' in g.es.attributes() else None).as_clustering()
    membership = communities.membership
    import matplotlib
    cmap = matplotlib.cm.get_cmap('tab20')
    num_colors = cmap.N if hasattr(cmap, "N") else 20
    comm_vertex_colors = [matplotlib.colors.to_hex(cmap(i % num_colors)) for i in membership]
    g.vs["color"] = comm_vertex_colors
    # ノードの次数に応じてノードサイズを変更する
    degrees = g.betweenness()
    # 適度なスケーリングのために最小サイズと最大サイズを指定
    min_size = 20
    max_size = 80
    # スケーリング
    if degrees:
        min_deg = min(degrees)
        max_deg = max(degrees) if max(degrees) != min(degrees) else min(degrees) + 1
        # ノードサイズ算出
        scaled_sizes = [
            min_size + (deg - min_deg) / (max_deg - min_deg) * (max_size - min_size)
            for deg in degrees
        ]
    else:
        scaled_sizes = [min_size for _ in g.vs]
    g.vs["size"] = scaled_sizes

    # ノードサイズに基づき再描画（ノードの大きさ＝次数）
    fig, ax = plt.subplots(figsize=figsize)
    ig.plot(
        g,
        target=ax,
        layout=layout,
        vertex_size=g.vs["size"],
        vertex_label=g.vs["label"],
        vertex_label_size=vertex_label_size,
        vertex_color=g.vs["color"],
        edge_width=edge_widths,
        edge_color=edge_color,
        edge_curved=edge_curved,
        bbox=(figsize[0]*100, figsize[1]*100),
        margin=40,
    )
    plt.show()
    return g,communities

In [ ]:
# ノードのページランク指標に応じてノードサイズを変更
def plot_igraph_from_adjacency_pr(
    adjacency_matrix,
    labels,
    layout_type="fr",
    vertex_size=30,
    edge_width_scale=5,
    vertex_label_size=14,
    edge_curved=False,
    figsize=(8, 8),
    edge_threshold=0.0,
    vertex_color="skyblue",
    edge_color="gray",
    top_n_labels=20
):
    import igraph as ig
    import matplotlib.pyplot as plt
    import numpy as np

    # 隣接行列をnumpy配列に変換
    adj = np.array(adjacency_matrix)
    n = adj.shape[0]

    # エッジの重みが閾値以下のものは0にする
    adj = np.where(adj > edge_threshold, adj, 0)

    # igraphグラフ作成
    g = ig.Graph.Weighted_Adjacency(adj.tolist(), mode=ig.ADJ_UNDIRECTED, attr="weight", loops=False)
    g.vs["name"] = labels

    # 孤立ノード除去
    isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0.0]
    if isolated_nodes:
        g.delete_vertices(isolated_nodes)



    # ノードラベル
    # labels = vocab として渡される前提
    if "name" in g.vs.attributes():
        # g.vs["name"] から g.vs["label"] へコピー
        g.vs["label"] = g.vs["name"]
    else:
        # "name" がない場合のフォールバック
        g.vs["label"] = [str(i) for i in range(g.vcount())]
    # 次数の高いノードの上位top_n_labels件のみラベルを付与
    degrees = g.pagerank()
    top_indices = np.argsort(degrees)[-top_n_labels:]  # 上位top_n_labels件
    g.vs["label"] = [
        g.vs[i]["label"] if i in top_indices else "" for i in range(g.vcount())
    ]
    # レイアウト
    layout = g.layout(layout_type)

    # エッジの太さ
    edge_weights = g.es["weight"] if "weight" in g.es.attributes() else [1]*g.ecount()
    if len(edge_weights) > 0 and max(edge_weights) > 0:
        edge_widths = [edge_width_scale * (w / max(edge_weights)) for w in edge_weights]
    else:
        edge_widths = [1 for _ in edge_weights]

    # コミュニティ検出（例: Louvain法, fallback to fastgreedy if Louvain not available）
    try:
        communities = g.community_multilevel(weights=g.es['weight'] if 'weight' in g.es.attributes() else None)
    except AttributeError:
        # fallback method
        communities = g.community_fastgreedy(weights=g.es['weight'] if 'weight' in g.es.attributes() else None).as_clustering()
    membership = communities.membership
    import matplotlib
    cmap = matplotlib.cm.get_cmap('tab20')
    num_colors = cmap.N if hasattr(cmap, "N") else 20
    comm_vertex_colors = [matplotlib.colors.to_hex(cmap(i % num_colors)) for i in membership]
    g.vs["color"] = comm_vertex_colors
    # ノードの次数に応じてノードサイズを変更する
    degrees = g.pagerank()
    # 適度なスケーリングのために最小サイズと最大サイズを指定
    min_size = 20
    max_size = 80
    # スケーリング
    if degrees:
        min_deg = min(degrees)
        max_deg = max(degrees) if max(degrees) != min(degrees) else min(degrees) + 1
        # ノードサイズ算出
        scaled_sizes = [
            min_size + (deg - min_deg) / (max_deg - min_deg) * (max_size - min_size)
            for deg in degrees
        ]
    else:
        scaled_sizes = [min_size for _ in g.vs]
    g.vs["size"] = scaled_sizes

    # ノードサイズに基づき再描画（ノードの大きさ＝次数）
    fig, ax = plt.subplots(figsize=figsize)
    ig.plot(
        g,
        target=ax,
        layout=layout,
        vertex_size=g.vs["size"],
        vertex_label=g.vs["label"],
        vertex_label_size=vertex_label_size,
        vertex_color=g.vs["color"],
        edge_width=edge_widths,
        edge_color=edge_color,
        edge_curved=edge_curved,
        bbox=(figsize[0]*100, figsize[1]*100),
        margin=40,
    )
    plt.show()
    return g,communities

In [ ]:
# コミュニティ検出なしでネットワーク構築
def plot_network_nocom(
    adjacency_matrix,
    labels,
    layout_type="fr",
    vertex_size=30,
    edge_width_scale=5,
    vertex_label_size=14,
    edge_curved=False,
    figsize=(8, 8),
    edge_threshold=0.0,
    vertex_color="skyblue",
    edge_color="gray",
    top_n_labels=20
):
    import igraph as ig
    import matplotlib.pyplot as plt
    import numpy as np

    # 隣接行列をnumpy配列に変換
    adj = np.array(adjacency_matrix)
    n = adj.shape[0]

    # エッジの重みが閾値以下のものは0にする
    adj = np.where(adj > edge_threshold, adj, 0)

    # igraphグラフ作成
    g = ig.Graph.Weighted_Adjacency(adj.tolist(), mode=ig.ADJ_UNDIRECTED, attr="weight", loops=False)
    g.vs["name"] = labels

    # 孤立ノード除去
    isolated_nodes = [v.index for v in g.vs if g.degree(v) == 0.0]
    if isolated_nodes:
        g.delete_vertices(isolated_nodes)



    # ノードラベル
    # labels = vocab として渡される前提
    if "name" in g.vs.attributes():
        # g.vs["name"] から g.vs["label"] へコピー
        g.vs["label"] = g.vs["name"]
    else:
        # "name" がない場合のフォールバック
        g.vs["label"] = [str(i) for i in range(g.vcount())]
    # 次数の高いノードの上位top_n_labels件のみラベルを付与
    degrees = g.degree()
    top_indices = np.argsort(degrees)[-top_n_labels:]  # 上位top_n_labels件
    g.vs["label"] = [
        g.vs[i]["label"] if i in top_indices else "" for i in range(g.vcount())
    ]
    # レイアウト
    layout = g.layout(layout_type)

    # エッジの太さ
    edge_weights = g.es["weight"] if "weight" in g.es.attributes() else [1]*g.ecount()
    if len(edge_weights) > 0 and max(edge_weights) > 0:
        edge_widths = [edge_width_scale * (w / max(edge_weights)) for w in edge_weights]
    else:
        edge_widths = [1 for _ in edge_weights]

    # ノードの次数に応じてノードサイズを変更する
    degrees = g.degree()
    # 適度なスケーリングのために最小サイズと最大サイズを指定
    min_size = 20
    max_size = 80
    # スケーリング
    if degrees:
        min_deg = min(degrees)
        max_deg = max(degrees) if max(degrees) != min(degrees) else min(degrees) + 1
        # ノードサイズ算出
        scaled_sizes = [
            min_size + (deg - min_deg) / (max_deg - min_deg) * (max_size - min_size)
            for deg in degrees
        ]
    else:
        scaled_sizes = [min_size for _ in g.vs]
    g.vs["size"] = scaled_sizes

    # ノードサイズに基づき再描画（ノードの大きさ＝次数）
    fig, ax = plt.subplots(figsize=figsize)
    ig.plot(
        g,
        target=ax,
        layout=layout,
        vertex_size=g.vs["size"],
        vertex_label=g.vs["label"],
        vertex_label_size=vertex_label_size,
        vertex_color=vertex_color,
        edge_width=edge_widths,
        edge_color=edge_color,
        edge_curved=edge_curved,
        bbox=(figsize[0]*100, figsize[1]*100),
        margin=40,
    )
    plt.show()
    return g

In [ ]:
import pandas as pd
import numpy as np
import igraph as ig
from scipy import sparse

def get_node_metrics_fast(graph, communities):
    """
    行列演算を用いた高速版:
    全ノードに対して参加係数、z-score、Betweenness、PageRankを計算し、
    全件のDataFrameを返す（フィルタリングはしない）
    """
    # コミュニティ情報の取得
    membership = np.array(communities.membership)
    num_communities = len(communities)
    num_nodes = graph.vcount()

    # --- 高速化の前準備: 疎行列の作成 ---
    A = graph.get_adjacency_sparse()

    # U: 所属行列
    row_indices = np.arange(num_nodes)
    col_indices = membership
    data = np.ones(num_nodes)
    U = sparse.csr_matrix((data, (row_indices, col_indices)), shape=(num_nodes, num_communities))

    # K_dist: コミュニティ別次数行列
    K_dist = A @ U

    # k_i: 各ノードの全次数
    k_i = np.array(A.sum(axis=1)).flatten()

    # --- A. 参加係数 (Participation Coefficient) の計算 ---
    k_i_safe = np.where(k_i == 0, 1, k_i)
    K_dist_sq = K_dist.power(2)
    sum_k_is_sq = np.array(K_dist_sq.sum(axis=1)).flatten()
    participation_coeffs = 1.0 - (sum_k_is_sq / (k_i_safe ** 2))
    participation_coeffs[k_i == 0] = 0.0

    # --- B. コミュニティ内次数 (z-score) の計算 ---
    k_in = np.array(K_dist[np.arange(num_nodes), membership]).flatten()

    df_temp = pd.DataFrame({
        'comm_id': membership,
        'k_in': k_in
    })

    grouped = df_temp.groupby('comm_id')['k_in']
    means = grouped.transform('mean')
    stds = grouped.transform('std')
    stds = stds.replace(0, 1)

    z_scores = (df_temp['k_in'] - means) / stds
    z_scores = z_scores.fillna(0).values

    # --- C. データのまとめ ---

    # PageRank等の計算
    degree = k_i
    betweenness = graph.betweenness()
    pagerank = graph.pagerank()

    # 全ノードを含むDataFrameを作成
    df = pd.DataFrame({
        "name": graph.vs["name"],
        "community_id": membership, # ここにLouvainの結果(0, 1, 2...)が入ります
        "degree": degree,
        "pagerank": pagerank,
        "betweenness": betweenness,
        "z_score": z_scores,
        "participation": participation_coeffs
    })

    # ここではまだフィルタリングせず、全データを返します
    return df


def get_top10_per_community(graph, communities):
    """
    1. 全ノードの指標を一度に計算する
    2. 各コミュニティごとに、重要度（ここではdegree）が高い順に10件を抽出する
    """

    # 1. 全ノードの計算（ループは使いません）
    all_nodes_df = get_node_metrics_fast(graph, communities)

    # 2. Pandasの機能で「コミュニティごとに上位10件」を抽出
    #    ソート順位を変えたい場合は by="pagerank" などに変更してください
    final_df = (all_nodes_df
                .sort_values(["community_id", "degree"], ascending=[True, False])
                .groupby("community_id")
                .head(10)  # 各グループの上位10件を取得
                )

    return final_df,all_nodes_df

In [ ]:
def plot_network_scatter(df,
                         x_col="degree",
                         y_col="betweenness",
                         hue_col="community_id",
                         size_col="pagerank",
                         name_col="name",        # 追加: ノード名が入っているカラム名
                         label_threshold_x=None,
                         figsize=(12, 8)):
    """
    ネットワーク指標の散布図を描画し、nameカラムの値を使ってラベルを付ける関数
    """

    # 図の準備
    plt.figure(figsize=figsize)

    # community_idなどが数値の場合、カテゴリとして扱うために文字列に変換
    plot_df = df.copy()
    if hue_col in plot_df.columns:
        plot_df[hue_col] = plot_df[hue_col].astype(str)

    # 散布図の描画
    sns.scatterplot(
        data=plot_df,
        x=x_col,
        y=y_col,
        hue=hue_col,
        size=size_col,
        sizes=(50, 600),
        alpha=0.7,
        palette="tab10",
        edgecolor="black"
    )

    # --- ラベルの付与処理 (修正箇所) ---
    if label_threshold_x is not None:
        # 閾値条件を満たす行だけ抽出
        labels_df = df[df[y_col] >= label_threshold_x]

        # 行ごとにループ処理 (indexではなく、rowから情報を取得)
        for _, row in labels_df.iterrows():
            plt.text(
                x=row[x_col],
                y=row[y_col],
                s=row[name_col],  # 修正: 指定したカラム(name)の値を使用
                fontsize=13,
                color='black',
                fontweight='bold',
                ha='right',
                va='bottom'
            )

    # グリッドやタイトルの設定
    plt.title(f"Network Metrics: {x_col} vs {y_col}", fontsize=20)
    plt.xlabel(x_col, fontsize=20)
    plt.ylabel(y_col, fontsize=20)
    plt.grid(True, linestyle='--', alpha=0.6)

    # 凡例を枠外に出す
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0,fontsize=20)

    plt.tight_layout()
    plt.show()

### 実行

In [ ]:
tt_df = pd.read_csv("h2o/matrix/tt_loyal.csv")
tt_= tt_df.to_numpy()

In [ ]:
adj_matrix = adj_matrix_(tt_,vocab,threshold=0)
find_threshold_by_count(adj_matrix, target_edge_count=900)

In [ ]:
g,communities = plot_igraph_from_adjacency_pr(adj_matrix,labels=vocab,edge_threshold=0.0006503,top_n_labels=15,vertex_label_size=8,vertex_size=50)
community_top10,all_nodes_df = get_top10_per_community(g, communities)

In [ ]:
plot_network_scatter(all_nodes_df, x_col="degree", y_col="betweenness", hue_col="community_id", size_col="pagerank",name_col="name", label_threshold_x=2000, figsize=(12, 8))